In [1]:
import datetime as dt

### Question 1

In [2]:
import pandas as pd

In [3]:
# This shows the version of pandas

pd.__version__

'0.24.2'

### Question 2

In [4]:
a = pd.read_csv("Csv1.csv")


a["LocationMilestones.OpenDate"] = pd.to_datetime(a["LocationMilestones.OpenDate"]).dt.date
a["LocationMilestones.LastRemodelDate"] = pd.to_datetime(a["LocationMilestones.LastRemodelDate"], errors = 'coerce').dt.date

a["TimeZone.TimeZoneCode"]= a["TimeZone.TimeZoneCode"].astype("category")

In [5]:
b = pd.read_csv("Csv2.csv", parse_dates= ["BeginTime.MF","ThruTime.MF"])

b["BeginTime.MF"] = b["BeginTime.MF"].dt.time
b["ThruTime.MF"] = b["ThruTime.MF"].dt.time


### Question 3

In [6]:
# Merging Dataframes a and b on the "ID" column: Name and Address.City was added because the columns were identical.

c = a.merge(b, how="inner", on = ["ID", "Name", "Address.City"], sort = True)

### Question 4

In [7]:
# Getting the number of rows

len(c)

108

In [8]:
# Getting the number of columns

len(c.columns)

13

In [9]:
# Getting the datatypes 

c.dtypes

Address.City                            object
ID                                       int64
LocationMilestones.LastRemodelDate      object
LocationMilestones.OpenDate             object
Name                                    object
Store.StoreDistrictID                    int64
TimeZone.TimeZoneCode                 category
PhoneNumber                             object
FaxNumber                               object
BeginTime.MF                            object
ThruTime.MF                             object
IsOpen.Sat                                bool
IsOpen.Sun                                bool
dtype: object

In [10]:
# Getting the summary

c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 0 to 107
Data columns (total 13 columns):
Address.City                          108 non-null object
ID                                    108 non-null int64
LocationMilestones.LastRemodelDate    73 non-null object
LocationMilestones.OpenDate           108 non-null object
Name                                  108 non-null object
Store.StoreDistrictID                 108 non-null int64
TimeZone.TimeZoneCode                 108 non-null category
PhoneNumber                           108 non-null object
FaxNumber                             103 non-null object
BeginTime.MF                          108 non-null object
ThruTime.MF                           108 non-null object
IsOpen.Sat                            108 non-null bool
IsOpen.Sun                            108 non-null bool
dtypes: bool(2), category(1), int64(2), object(8)
memory usage: 9.8+ KB


In [11]:
# The numpy array of the dataframe

c.values

array([['Madison', 144, datetime.date(2007, 3, 23), ...,
        datetime.time(22, 0), True, True],
       ['Nashville', 146, datetime.date(2012, 3, 22), ...,
        datetime.time(23, 0), True, True],
       ['Knoxville', 151, datetime.date(2014, 4, 3), ...,
        datetime.time(23, 0), True, False],
       ...,
       ['Homewood', 2796, NaT, ..., datetime.time(23, 0), True, False],
       ['Knoxville', 2815, NaT, ..., datetime.time(22, 0), False, True],
       ['Madison', 2816, NaT, ..., datetime.time(22, 0), True, True]],
      dtype=object)

In [12]:
# The list equivalent of the dataframe

list(c)

['Address.City',
 'ID',
 'LocationMilestones.LastRemodelDate',
 'LocationMilestones.OpenDate',
 'Name',
 'Store.StoreDistrictID',
 'TimeZone.TimeZoneCode',
 'PhoneNumber',
 'FaxNumber',
 'BeginTime.MF',
 'ThruTime.MF',
 'IsOpen.Sat',
 'IsOpen.Sun']

### Question 5

In [13]:
# Renaming the column

c.rename(columns = {"Name": "Store Name"}, inplace = True)

### Question 6

In [14]:
# Checking for null values

c.isnull().sum()

Address.City                           0
ID                                     0
LocationMilestones.LastRemodelDate    35
LocationMilestones.OpenDate            0
Store Name                             0
Store.StoreDistrictID                  0
TimeZone.TimeZoneCode                  0
PhoneNumber                            0
FaxNumber                              5
BeginTime.MF                           0
ThruTime.MF                            0
IsOpen.Sat                             0
IsOpen.Sun                             0
dtype: int64

### Question 7

In [15]:
w = c["IsOpen.Sat"].value_counts()

In [16]:
x = c["IsOpen.Sun"].value_counts()

In [32]:
w

True     101
False      7
Name: IsOpen.Sat, dtype: int64

In [33]:
x

True     86
False    22
Name: IsOpen.Sun, dtype: int64

In [34]:
from math import pi

from bokeh.layouts import row
from bokeh.models import HoverTool
from bokeh.io import output_file, show, output_notebook
from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook()

Loading BokehJS ...

In [39]:
data = w.reset_index(name='count').rename(columns={'index':'Is_Open'})
data['angle'] = data['count']/sum(w.values) * 2*pi
data['color'] = ['violet', 'gray']
data['percentage'] = data['count']/sum(w.values) * 100



p = figure(plot_height=350, title="Stores Availability on Saturdays (in %)", toolbar_location='right')

hover = HoverTool(tooltips = [('Is Open','@Is_Open'),('How Many','@count'), ('Percentage','@percentage')])
p.add_tools(hover)

p.title.align = "center"
p.title.text_font_size = "20px"

p.wedge(x=0, y=1, radius=0.4, source = data,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", legend='Is_Open', fill_color='color')

data = x.reset_index(name='count').rename(columns={'index':'Is_Open'})
data['angle'] = data['count']/sum(x.values) * 2*pi
data['color'] = ['green', 'olive']
data['percentage'] = data['count']/sum(x.values) * 100

pc = figure(plot_height=350, title="Stores Availability on Sundays (in %)", toolbar_location='right')
pc.wedge(x=0, y=1, radius=0.4, source = data,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", legend='Is_Open', fill_color='color')

hover = HoverTool(tooltips = [('Is Open','@Is_Open'),('How Many','@count'), ('Percentage','@percentage')])
pc.add_tools(hover)

pc.title.align = "center"
pc.title.text_font_size = "20px"

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

pc.axis.axis_label=None
pc.axis.visible=False
pc.grid.grid_line_color = None


f = row(p,pc)
show(f)

### Question 8

In [ ]:
data

In [ ]:
mask2 = c["ThruTime.MF"]== dt.time(23)

eleven_pm_stores = c[mask2]["Store Name"].to_list()

In [ ]:
eleven_pm_stores

### Question 9

In [ ]:
# list of stores remodelled and remodel date

mask1 = c["LocationMilestones.LastRemodelDate"].notnull()

# list of stores that were remodeled
store_names = c[mask1]["Store Name"].to_list()

# list of store remodel dates
remodel_dates = c[mask1]["LocationMilestones.LastRemodelDate"].to_list()

In [ ]:
remodeled_stores = {}

for i,j in zip(store_names, remodel_dates):
    remodeled_stores[i] = j
    

In [ ]:
remodeled_stores

### Question 10

In [ ]:
# Grouping the dataframe by Address City

d = c.groupby("Address.City")

In [ ]:
# Count of occurence
d.size()

### Question 11

In [ ]:
time_range = pd.date_range('2018-01-01', periods=10, freq='YS')

In [ ]:
time_series = pd.Series(time_range)

In [ ]:
time_series

In [ ]:
c.head()